In [ ]:
# Cell 1 - IMPORT

# fcc_cat_dog.ipynb
#try:
  # This command only in Colab.
#  %tensorflow_version 2.x
#except Exception:
#  pass
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os          #needed to download the file
import zipfile   #I need this on my local computer for opening the file
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Cell 2 - LOAD FILES

# Get project files
!wget https://cdn.freecodecamp.org/project-data/cats-and-dogs/cats_and_dogs.zip

!unzip cats_and_dogs.zip

PATH = 'cats_and_dogs'

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
test_dir = os.path.join(PATH, 'test')

# Get number of files in each directory. The train and validation directories
# each have the subdirecories "dogs" and "cats".
total_train = sum([len(files) for r, d, files in os.walk(train_dir)])
total_val = sum([len(files) for r, d, files in os.walk(validation_dir)])
total_test = len(os.listdir(test_dir))

# Variables for pre-processing and training.
batch_size = 128
epochs = 15
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
# 3 - LOAD DATA IN SUITABLE FORMAT
# Using the Keras/TensorFlow ImageGenerator tool. In this case, it is being used to:
# * Apply transformations, e.g., rescale the pixel values of images.
# * Load images from directories into batches.
# * Label images based on directory structure (e.g., cats and dogs).

# Normalising images from [0, 255] to [0, 1]
train_image_generator = ImageDataGenerator(rescale=1./255)          # Used to preprocess and load the training images
validation_image_generator = ImageDataGenerator(rescale=1./255)     # Used for the validation dataset; here validation images are processed the same way as training images.
test_image_generator = ImageDataGenerator(rescale=1./255)           # Used for the test dataset (preprocesses images before making predictions).

# Loading training images using flow_from_directory
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,     
    directory=train_dir,                                         # training directory (has subfolders cats and dogs)
    target_size=(IMG_HEIGHT, IMG_WIDTH),                         # resizing of all images (from cell 2 we see this is 150 x 150)
    class_mode='binary'                                          # binary labels: 0 and 1 (cats & dogs)
)
# as above, but now for validation data
val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

# As above, but now for test data
# Note: test directory does not have subdirectories, so we specify 'class_mode=None' and 'shuffle=False'
test_data_gen = test_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode=None,
    shuffle=False,
    classes=['test']
)


In [ ]:
# CELL 4 - PLOT IMAGES
# Creating the function "plotImages" to display images in a vertical column.
# %matplotlib inline

def plotImages(images_arr, probabilities = False):         # 2 inputs: images_arr - A list or array of images to be displayed; probabilities (optional) - corresponding list of probabilities (labels of cat/dog probabilities shown on images if True)
    fig, axes = plt.subplots(len(images_arr), 1, figsize=(5,len(images_arr) * 5))   # creating a grid of subplots, with the number of images being len(images_arr) and 1 image per row; figsize defined to clearly see the images
    if probabilities is False:                      # if the input probabilities is False....
      for img, ax in zip( images_arr, axes):        
          ax.imshow(img)                            # shows all images
          ax.axis('off')                            # axis lines, ticks, & labels hidden (for clearer view)
    else:
      for img, probability, ax in zip( images_arr, probabilities, axes): # if probabilities = True....
          ax.imshow(img)                                                 # show all iamges
          ax.axis('off')                                                 # hides the axis lines, ticks, and labels for a cleaner image display
          if probability > 0.5:                                          # if probability above 0.5....
              ax.set_title("%.2f" % (probability*100) + "% dog")         # title of image set to *probability % dog*...
          else:                                                          # else
              ax.set_title("%.2f" % ((1-probability)*100) + "% cat")     # title of image set to *probability % cat*
    plt.show()                                                           # show the plot (with all the subplots)

sample_training_images, _ = next(train_data_gen)           #batch of images, labels = next(train_data_gen) retrieves a batch of images from the training data generator (labels ignored here; we are only displaying images)
plotImages(sample_training_images[:5])                     # using the function; first 5 images of the batch are shown

In [ ]:
# 5 - DATA AUGMENTATION
# In order to get a more varied dataset, we are going to create a function that modifies our existing images so that it looks like we have more images
train_image_generator = ImageDataGenerator(       # using the ImageDataGenerator tool again for:
    rescale=1./255,                               # normalising images ([0,255] to [0,1])
    rotation_range=40,                            # randomly rotate images by 40 degrees 
    width_shift_range=0.2,                        # randomly width shift images by 20%
    height_shift_range=0.2,                       # randomly height shift images by 20%
    shear_range=0.2,                              # randomly shear shift images by 20%
    zoom_range=0.2,                               # randomly zoom into images by 20%
    horizontal_flip=True,                         # randomly flip images horizontally
    fill_mode='nearest'                           # fill in gaps (due to modifications) with "nearest" pixels
)

In [ ]:
# 6 - VISUALISING DATA AUGMENTATION
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,                # creating a data generator that: transforms the images (using the above function)  after having
                                                     directory=train_dir,                        # loaded images from train_dir (in specific batch sizes) into "directory" (there are 2 subfolders: cats, dogs)
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),        # images of specific dimensions (150 x 150)
                                                     class_mode='binary')                        # labels classed as binary (0 and 1) 

augmented_images = [train_data_gen[0][0][0] for i in range(5)]                                   #  train_data_gen[0]: Gets the first batch of images and labels.
                                                                                                 # [0]: Extracts the images from the batch (ignores the labels).
                                                                                                 # [0]: Retrieves the first image in the batch.
# This is repeated 5 times to get 5 differently augmented versions of the same image.
# Each time the image is loaded, a new transformation is applied, resulting in 5 variations.

plotImages(augmented_images)                                                                     # Use the plotImages function from before to plot augmented images

In [ ]:
# 7 - BUILDING & COMPLILING THE MODEL

# BUILDING THE MODEL
# creating the CNN using the Keras Sequential API. Lower layers learn basic features (edges, corners), higher layers learn more complex patters (faces, etc.)
model = Sequential([                                                                  # layers are to be stacked sequentially
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),     # 1st convolution block: 32 filters (feature detectors), each having a size of (3x3); Rectified Linear Unit is applied to introduce non-linearity. 
                                                                                      # input size is 150 x 150 x 3 (3 here being the channels, so it is RGB)
    MaxPooling2D(pool_size=(2, 2)),                                                   # use pooling to maximise computational efficiency; here the feature map is a 2x2 window
    
    Conv2D(64, (3,3), activation='relu'),                                             # 2nd convolution block; as above, but now with 64 filters (to detect more complex patterns)
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3,3), activation='relu'),                                            # 3rd convolution block; now with 128 filters (even more depth)
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3,3), activation='relu'),                                            # 4th convolution block; again, with 128 filters
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),                                                                        # From 2D feature map to 1D vector
    Dense(512, activation='relu'),                                                    # dense (fully connected) layer of 128*4 = 512 neurons, with ReLU activation
    Dropout(0.5),                                                                     # Randomly set 50% of the neurons to zero during training (prevents overfitting by making the model less sensitive to specific neurons.)
    Dense(1, activation='sigmoid')                                                    # Output layer: 1 dense neuron (b/c binary classification), using sigmoid activation (0 to 1)
])


# COMPILING THE MODEL
model.compile(optimizer='adam',                                                       # Using Adaptive Moment Estimation (adam) optimisation (combines the advantages of RMSProp and Momentum)
              loss='binary_crossentropy',                                             # applying crossentropy to loss (measures the difference between predicted probabilities and actual labels)
              metrics=['accuracy'])                                                   # Tracks accuracy during training and validation, and displays it after each epoch


model.summary()                                                                       # outputs a summary of the model's features


In [ ]:
# 8 - TRAINING THE MODEL (app. 20min on a Macbook)
# Here we are specifying the number of epochs, the steps per epoch and validation steps, and we are training and validation data generators.
history = model.fit(                               # Keras "fit" is used to feed batches of data to the model, calculate loss, and perform backpropagation (update weights)
    train_data_gen,                                # function (created in cell 6) for augmentated data
    steps_per_epoch=total_train // batch_size,     # number of batches per epoch = total no. of training images / no. of images per batch (Note: use // to get an integer number) 2000 // 128 = 15 batches per epoch
    epochs=epochs,                                 # no of epochs as defined before
    validation_data=val_data_gen,                  # val_data_gen: no augmentation. This only loads validation images, rescales the pixels, and uses binary labels for cats and dogs (for evaulation of the model on unseen data)
    validation_steps=total_val // batch_size       # batches for validation set = 1000 // 128 = 7
)

In [ ]:
# 9 - VISUALISATION OF MODEL'S PERFORMANCE
# plotting: Training and Validation Accuracy; Training and Validation Loss

acc = history.history['accuracy']               # history.history contains the training statistics recorded during .fit() in Cell 8 (stored as lists; one entry per epoch)
val_acc = history.history['val_accuracy']       # validation accuracy for each epoch

loss = history.history['loss']                  # training loss for each epoch
val_loss = history.history['val_loss']          # validation loss for each epoch

epochs_range = range(epochs)                    # creating a sequence from 0 to (epoch no. - 1); used for plotting x-axis

plt.figure(figsize=(10, 5))                                           # size of whole plot is 10 x 5
plt.subplot(1, 2, 1)                                                # 1st subplot (training and validation accuracy): 1 row, 2 columns, position 1.                           
plt.plot(epochs_range, acc, label='Training Accuracy')              # Plot training accuracy against epochs.
plt.plot(epochs_range, val_acc, label='Validation Accuracy')        # Plot validation accuracy against epochs.
plt.legend(loc='lower right')                                       # Add a legend (to distinguish training and validation curves)
plt.title('Training and Validation Accuracy')                       # add a subplot title

plt.subplot(1, 2, 2)                                                # 2nd subplot (training and validation loss): 1 row, 2 columns, position 2. 
plt.plot(epochs_range, loss, label='Training Loss')                 # plot training loss against epochs
plt.plot(epochs_range, val_loss, label='Validation Loss')           # plot validation loss against epochs
plt.legend(loc='upper right')                                       # add a legend 
plt.title('Training and Validation Loss')                           # add a subplot title
plt.show()                                                          # show the whole plot 

In [ ]:
# 10 - PREDICTONS ON TEST DATA
# predicts the probability of each image being a dog (> 0.5) or cat (<0.5), and plots the images with their predicted labels.

# print(test_data_gen.filepaths) # checking to see if the filepath to test_data_gen data is empty;

probabilities = model.predict(test_data_gen)          # Feeds all test images through the trained model and returns an array of predicted probabilities (close to 1 = dog, close to 0 = cat)
probabilities = [prob[0] for prob in probabilities]   # Flatten the list; the above returns a list of lists; this line of code makes things more simpler (easier to work with 1D list)

test_images = [test_data_gen[0][0][i] for i in range(50)] # test_data_gen[0]: Gets the first batch from the test generator, [0]: Selects the images (no labels for test test), [i]: Retrieves each image in the batch
                                                          # We use 50 images (which is actually the total of images here)

print(np.array(test_images).shape)
plotImages(test_images, probabilities)                    # use plotImage function (from cell 4) to display images in a vertical column, and show the predicted label and confidence percentage for each image

In [ ]:
# 11 - CHECKING RESULTS
# calculates the percentage of correctly classified images

answers =  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
            1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
            1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
            1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 
            0, 0, 0, 0, 0, 0]

correct = 0

for probability, answer in zip(probabilities, answers):
  if round(probability) == answer:
    correct +=1

percentage_identified = (correct / len(answers)) * 100

passed_challenge = percentage_identified >= 63

print(f"Your model correctly identified {round(percentage_identified, 2)}% of the images of cats and dogs.")

if passed_challenge:
  print("You passed the challenge!")
else:
  print("You haven't passed yet. Your model should identify at least 63% of the images. Keep trying. You will get it!")